# Imports

In [2]:
import torch
import numpy as np
import torch.nn as nn

import matplotlib.pyplot as plt

# Model

## Sizes



- x : (batch_size, max_length) 

- tokens_id in x between 0 and vocab_size

- Embedd(x) : (batch_size, max_length, model_dim)

- K : (model_dim, dk)
- Kx : (batch_size, max_length, dk)

- Q : (model_dim, dk)
- Qx : (batch_size, max_length, dk)

- Qx*Kx^T : (batch_size, max_length, max_length)
- V : (model_dim, dv)
- Vx : (batch_size, max_length, dv)





## Implementation

In [3]:


class PositionalEncoding(nn.Module):
    def __init__(self, batch_size, model_dim, max_length):
        super().__init__()
        self.model_dim = model_dim
        self.max_length = max_length
        self.batch_size = batch_size
        self.compute()

    def SinPos(self, i: int, pos: int):
        if i % 2 == 0:
            return np.sin(pos / 10000 ** (2 * i / self.model_dim))
        else:
            return np.cos(pos / 10000 ** (2 * i / self.model_dim))

    def compute(self):
        Mat = torch.Tensor([[self.SinPos(i, pos) for i in range(self.model_dim)] for pos in range(self.max_length)])
        self.Mat = Mat.expand(self.batch_size, -1, -1)

    def forward(self, x):
        with torch.no_grad():
            return self.Mat


class Embedding(nn.Module):
    def __init__(self, batch_size, model_dim, max_length, n_embedding):
        super().__init__()
        self.max_length = max_length
        self.batch_size = batch_size
        self.model_dim = model_dim
        self.n_embedding = n_embedding
        self.embedding = torch.nn.Embedding(num_embeddings=n_embedding, embedding_dim=model_dim)
        self.pos_encoding = PositionalEncoding(batch_size=batch_size, model_dim=model_dim, max_length=max_length)
        pass

    def forward(self, x):
        x = self.embedding(x) + self.pos_encoding(x)
        return x


class SingleHeadAttention(nn.Module):
    def __init__(self, dk: int, dv: int, model_dim: int, mask: torch.Tensor = None):
        super().__init__()
        self.dk = dk
        self.dv = dv
        self.model_dim = model_dim
        self.K = nn.Linear(in_features=model_dim, out_features=dk)
        self.Q = nn.Linear(in_features=model_dim, out_features=dk)
        self.V = nn.Linear(in_features=model_dim, out_features=dv)
        self.mask = mask

    def forward(self, x: torch.Tensor, x_encoder: torch.Tensor = None):
        Kx = self.K(x_encoder) if x_encoder is not None else self.K(x)
        Vx = self.V(x_encoder) if x_encoder is not None else self.V(x)
        Qx = self.Q(x)
        QK = torch.matmul(Qx, Kx.transpose(-2, -1)) / np.sqrt(self.dk)
        if self.mask is not None:
            QK += self.mask
        QK = torch.softmax(QK, dim=-1)
        return torch.matmul(QK, Vx)


class MultiHeadAttention(nn.Module):
    def __init__(self, num_heads: int, dk: int, dv: int, model_dim: int, mask=None):
        super().__init__()
        assert num_heads * dv == model_dim, "num_heads * dv should be equal to the model dim"
        self.attention_heads = nn.ModuleList([SingleHeadAttention(dk=dk, dv=dv, model_dim=model_dim, mask=mask) for _ in range(num_heads)])
        self.WO = nn.Linear(in_features=num_heads * dv, out_features=model_dim)
        self.mask = mask

    def forward(self, x: torch.Tensor, x_encoder: torch.Tensor = None):
        outputs = [head(x, x_encoder) for head in self.attention_heads]
        x = torch.cat(outputs, dim=-1)
        x = self.WO(x)
        return x


class EncoderBlock(nn.Module):
    def __init__(self, num_heads, dk, dv, d_ff, model_dim, dropout):
        super().__init__()
        self.attention = MultiHeadAttention(num_heads=num_heads, dk=dk, dv=dv, model_dim=model_dim)
        self.layerNorm1 = nn.LayerNorm(normalized_shape=model_dim)
        self.layerNorm2 = nn.LayerNorm(normalized_shape=model_dim)
        self.ff = nn.Sequential(
            nn.Linear(in_features=model_dim, out_features=d_ff),
            nn.ReLU(),
            nn.Dropout(p=dropout),
            nn.Linear(in_features=d_ff, out_features=model_dim),
        )

    def forward(self, x):
        attention = self.attention(x)
        x = self.layerNorm1(x + attention)
        feedforward = self.ff(x)
        x = self.layerNorm2(x + feedforward)
        return x


class Encoder(nn.Module):
    def __init__(self, num_heads, dk, dv, d_ff, model_dim, dropout, num_encoders):
        super().__init__()
        self.encoders_list = [
            EncoderBlock(num_heads=num_heads, dk=dk, dv=dv, d_ff=d_ff, model_dim=model_dim, dropout=dropout) for _ in range(num_encoders)
        ]
        self.encoders = nn.Sequential(*self.encoders_list)

    def forward(self, x):
        x = self.encoders(x)
        return x


class DecoderBlock(nn.Module):
    def __init__(self, num_heads, dk, dv, d_ff, model_dim, dropout, max_length):
        super().__init__()
        self.mask = torch.zeros(max_length, max_length) + torch.triu(torch.full((max_length, max_length), float("-inf")), diagonal=1)
        self.masked_attention = MultiHeadAttention(num_heads=num_heads, dk=dk, dv=dv, model_dim=model_dim, mask=self.mask)
        self.mixed_attention = MultiHeadAttention(num_heads=num_heads, dk=dk, dv=dv, model_dim=model_dim)
        self.layerNorm1 = nn.LayerNorm(normalized_shape=model_dim)
        self.layerNorm2 = nn.LayerNorm(normalized_shape=model_dim)
        self.layerNorm3 = nn.LayerNorm(normalized_shape=model_dim)

        self.ff = nn.Sequential(
            nn.Linear(in_features=model_dim, out_features=d_ff),
            nn.ReLU(),
            nn.Dropout(p=dropout),
            nn.Linear(in_features=d_ff, out_features=model_dim),
        )

    def forward(self, x, x_encoder):
        attention = self.masked_attention(x)
        x = self.layerNorm1(x + attention)
        attention = self.mixed_attention(x, x_encoder)
        x = self.layerNorm2(x + attention)
        feedforward = self.ff(x)
        x = self.layerNorm3(x + feedforward)
        return x


class CustomSequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        self.modules_list = nn.ModuleList(args)

    def forward(self, x, x_encoder):
        for module in self.modules_list:
            x = module(x, x_encoder)
        return x


class Decoder(nn.Module):
    def __init__(self, num_heads, dk, dv, d_ff, model_dim, max_length, dropout, num_decoders):
        super().__init__()
        decoders_list = [
            DecoderBlock(
                num_heads=num_heads,
                dk=dk,
                dv=dv,
                d_ff=d_ff,
                model_dim=model_dim,
                dropout=dropout,
                max_length=max_length,
            )
            for _ in range(num_decoders)
        ]
        self.decoders = CustomSequential(*decoders_list)

    def forward(self, x, x_encoder):
        x = self.decoders(x, x_encoder)
        return x


class Transformer(nn.Module):
    def __init__(
        self,
        batch_size,
        model_dim,
        max_length,
        vocab_size,
        num_out,
        num_heads,
        dv,
        dk,
        d_ff,
        dropout,
        num_encoders,
        num_decoders,
    ):
        super().__init__()
        self.encoder = Encoder(
            num_heads=num_heads,
            dk=dk,
            dv=dv,
            d_ff=d_ff,
            model_dim=model_dim,
            dropout=dropout,
            num_encoders=num_encoders,
        )
        self.decoder = Decoder(
            num_heads=num_heads,
            dk=dk,
            dv=dv,
            d_ff=d_ff,
            model_dim=model_dim,
            dropout=dropout,
            num_decoders=num_decoders,
            max_length=max_length,
        )
        self.linear = nn.Linear(in_features=model_dim, out_features=num_out)
        self.embedding = Embedding(
            batch_size=batch_size,
            model_dim=model_dim,
            max_length=max_length,
            n_embedding=vocab_size,
        )

    def forward(self, x):
        x = self.embedding(x)
        x_encoder = self.encoder(x)
        x = self.decoder(x, x_encoder)
        x = self.linear(x)
        return x


# Tests

In [ ]:
batch_size = 16
model_dim = 512
max_length = 100
vocab_size = 20000
num_out = vocab_size
num_heads = 8
dv = 64
dk = 64
d_ff = 2048
dropout = 0.1
num_encoders = 6
num_decoders = 6




def init_transformer():
    model = Transformer(
        batch_size=batch_size,
        model_dim=model_dim,
        max_length=max_length,
        vocab_size=vocab_size,
        num_out=num_out,
        num_heads=num_heads,
        dv=dv,
        dk=dk,
        d_ff=d_ff,
        dropout=dropout,
        num_encoders=num_encoders,
        num_decoders=num_decoders,
    )
    for p in model.parameters():
        if p.dim() > 1:
            nn.init.xavier_uniform_(p)
    return model

def inference(model, input, max_gen_length):
    output = input
    x = input
    for i in range(max_gen_length):
        proba = torch.softmax(model(x), dim=-1)
        max_proba, next_token = torch.max(proba, dim=-1)
        next_token = next_token[:, -1].unsqueeze(-1)
        x = torch.cat((x[:, 1:], next_token), dim=1)
        output = torch.cat((output, next_token), dim=1)
    return output


x = torch.randint(0, vocab_size, (batch_size, max_length))
MyTransformer = init_transformer()




inference(MyTransformer, x, 10).shape

torch.Size([16, 110])

## Tokenizer

In [98]:
from tokenizers import Tokenizer
from tokenizers.models import BPE
from tokenizers.trainers import BpeTrainer
from tokenizers.pre_tokenizers import Whitespace

tokenizer = Tokenizer(BPE())

tokenizer.pre_tokenizer = Whitespace()
special_tokens = ["[PAD]", "[UNK]", "[CLS]", "[SEP]", "[MASK]"]
trainer = BpeTrainer(special_tokens=special_tokens)

tokenizer.train(["data.txt"], trainer)
tokenizer.save("bpe_tokenizer.json")

tokenizer = Tokenizer.from_file("bpe_tokenizer.json")

with open('/users/eleves-b/2021/valentin.dorseuil/Desktop/ScratchML/transformers/data.txt', 'r') as file:
    lines = file.readlines()


chunk_size = 10
overlap_size = 5
chunks = [''.join(lines[i:i+chunk_size]) for i in range(0, len(lines) - chunk_size + 1, chunk_size - overlap_size)]

print(f"Vocabulary size: {tokenizer.get_vocab_size()}")

encoded_text = tokenizer.encode_batch(chunks)
for token in special_tokens:
    token_id = tokenizer.token_to_id(token)
    print(f"Token: {token}, ID: {token_id}")

print(f"Number of chunks: {len(encoded_text)}")
print(f"Average chunk length (num tokens): {sum([len(chunk) for chunk in encoded_text])/len(encoded_text):.2f}")
print()




Vocabulary size: 18151
Token: [PAD], ID: 0
Token: [UNK], ID: 1
Token: [CLS], ID: 2
Token: [SEP], ID: 3
Token: [MASK], ID: 4
Number of chunks: 7999
Average chunk length (num tokens): 65.50



## Data

In [101]:
tokenizer.decode(encoded_text[0].ids)

'First Citizen : Before we proceed any further , hear me speak . All : Speak , speak . First Citizen : You are all resolved rather to die than to famish ? All :'

In [99]:
import random

random.seed(123)
random.shuffle(chunks)

train_size = int(0.8 * len(chunks))
val_size = int(0.1 * len(chunks))
test_size = len(chunks) - train_size - val_size

train_chunks = chunks[:train_size]
val_chunks = chunks[train_size:train_size + val_size]
test_chunks = chunks[train_size + val_size:]